# Recurrent Neural Network

In [2]:
import numpy as np
import os

## keras
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, Activation, LSTM
from keras.utils import np_utils

# BasicRNNCell

---------------------------------------------------------------------------

## [Case1] seq2seq, Character "HelloWorld"

### 1. data define

In [3]:
sample = "HelloWorld"

char_set = list(set(sample))  
char_dic = {w: i for i, w in enumerate(char_set)}

char_set, char_dic

(['d', 'l', 'H', 'e', 'W', 'o', 'r'],
 {'H': 2, 'W': 4, 'd': 0, 'e': 3, 'l': 1, 'o': 5, 'r': 6})

In [4]:
x_str = sample[:-1]
y_str = sample[1:]

x_str, y_str

('HelloWorl', 'elloWorld')

In [21]:
# char to index
x = [char_dic[c] for c in x_str]  
y = [char_dic[c] for c in y_str]  

x, y

([2, 3, 1, 1, 5, 4, 5, 6, 1], [3, 1, 1, 5, 4, 5, 6, 1, 0])

### 2. Hyper-parameter

In [10]:
data_dim    = len(char_set)
timesteps   = len(y_str)
num_classes = len(char_set)

data_dim, timesteps, num_classes

(7, 9, 7)


## 3. One-hot Encoding + Data reshape

In [22]:
# One-hot encoding
x = np_utils.to_categorical(x, 7)
x, x.shape

(array([[ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  1.,  0.,  0.,  0.,  0.,  0.]]), (9, 7))

In [23]:
# reshape X to be [samples, time steps, features]
x = np.reshape(x, (-1, len(x), data_dim))
x, x.shape

(array([[[ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.]]]), (1, 9, 7))

In [24]:
y = np_utils.to_categorical(y, num_classes)
y = np.reshape(y, (-1, len(y), data_dim))
y, y.shape

(array([[[ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.]]]), (1, 9, 7))


### 4. Modeling

In [25]:
model = Sequential()

model.add(LSTM(num_classes, input_shape=(timesteps, data_dim), return_sequences=True))
model.add(TimeDistributed(Dense(num_classes)))
model.add(Activation('softmax'))
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 9, 7)          420         lstm_input_1[0][0]               
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 9, 7)          56          lstm_1[0][0]                     
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 9, 7)          0           timedistributed_1[0][0]          
Total params: 476
Trainable params: 476
Non-trainable params: 0
____________________________________________________________________________________________________


In [40]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [46]:
model.fit(x, y)

Epoch 1/10
1/1 [==============================] - 0s - loss: 1.8957 - acc: 0.3333
Epoch 2/10
1/1 [==============================] - 0s - loss: 1.8936 - acc: 0.3333
Epoch 3/10
1/1 [==============================] - 0s - loss: 1.8916 - acc: 0.4444
Epoch 4/10
1/1 [==============================] - 0s - loss: 1.8895 - acc: 0.4444
Epoch 5/10
1/1 [==============================] - 0s - loss: 1.8874 - acc: 0.4444
Epoch 6/10
1/1 [==============================] - 0s - loss: 1.8854 - acc: 0.4444
Epoch 7/10
1/1 [==============================] - 0s - loss: 1.8834 - acc: 0.5556
Epoch 8/10
1/1 [==============================] - 0s - loss: 1.8813 - acc: 0.5556
Epoch 9/10
1/1 [==============================] - 0s - loss: 1.8793 - acc: 0.5556
Epoch 10/10
1/1 [==============================] - 0s - loss: 1.8773 - acc: 0.5556


### 5. Predict Result

In [48]:
predictions = model.predict(x, verbose=0)


for i, prediction in enumerate(predictions):
    print(prediction)
    x_index = np.argmax(x[i], axis=1)
    x_str = [char_set[j] for j in x_index]
    print(x_index, ''.join(x_str))

    index = np.argmax(prediction, axis=1)
    result = [char_set[j] for j in index]
    print(index, ''.join(result))

[[ 0.138119    0.15028575  0.13049877  0.14729542  0.13618085  0.1555717
   0.14204855]
 [ 0.1153471   0.15938033  0.14261015  0.12842326  0.14264613  0.15495665
   0.15663634]
 [ 0.13742045  0.16306952  0.14247268  0.14297885  0.1270176   0.14857706
   0.13846391]
 [ 0.15058793  0.15709595  0.14808835  0.15046465  0.12566513  0.14014694
   0.12795103]
 [ 0.11739409  0.16999906  0.14862731  0.12610589  0.13213894  0.14847542
   0.15725927]
 [ 0.13519976  0.14922701  0.13355704  0.14002325  0.13627122  0.15840399
   0.14731775]
 [ 0.11456206  0.1705693   0.12964723  0.12413155  0.12766209  0.16286017
   0.17056765]
 [ 0.11580326  0.16755274  0.14261857  0.12380446  0.13978869  0.14946236
   0.16097002]
 [ 0.14576149  0.1681633   0.13770878  0.1452297   0.12160087  0.14168409
   0.13985166]]
[2 3 1 1 5 4 5 6 1] HelloWorl
[5 1 1 1 1 5 1 1 1] ollllolll



------------------------------------
*End of Code*